# InfluxDB / Docker

In this chapter we explain first the make-it-easy helper-script usage of InfluxDB time series database. In the second part we give necessary information for those who want to do the installation their way.

>**NOTE**: You are not obliged to run  InfluxDB time series database while underway - you can still record data in a file for off-line analysis. But the provided helper script method based on Docker makes it so easy that you do not want to miss the new services which can be built around having near-term historical data - all data - at your disposal, like on Grafana dashboards.

## Introduction

[Influx data](https://www.influxdata.com/)'s [InfluxDB 2.0](https://www.influxdata.com/products/influxdb-overview/influxdb-2-0/) is in final beta phase (_when this is written_) which allows us to develop a [connector](../idbout/idbout.ipynb#InfluxDB-Out) between _DashT_ OpenCPN 5 plug-in and this popular time-series database platform. Most natural usage is to export **all** data received by the plug-in into Influx DB 2.0. 

Once the all data is stored, it can be used to wide variety of usage, either immediately or extracted for off-line analysis. Here is an example of dashboards, familiar from Grafana but here provided by the InfluxDB 2.0 offering instant retrieval and analysis services for live data:

<img src="img/s_005_DashT_InfluxDB_Dashboard_and_3cells.png"
alt="InfluxDB dasboards" width="500">[(zoom)](img/005_DashT_InfluxDB_Dashboard_and_3cells.png)

## Docker InfluxDB

Docker, [Docker Desktop](https://www.docker.com/products/docker-desktop) for Windows and Mac is a popular framework to containerize applications. Shortly, a "container" is a mini-operating system, often Linux based running one single service, usually network based. The container is running under Docker or Docker desktop and you would communicate with the application over the network. The application can be made to see your file system so that you can share data with it without going through the network.

>**NOTE**: Why there is no _DashT_ Docker container? Simply because it would make yet another build to maintain and yet another dependency. Instead, standard and latest service containers can be used and configuration scripts for those are provided with _DashT_ installation package and explained below.

>**TIP**: You can try and *not* to stop InfluxDB using the below helper scripts when you stop your system; normally, at next restart Docker will restart all the services. But on Windows system, do not leave your system too long time off, without restarting the InfluxDB using the provided script - it synchronizes the clock only this way on Windows.

### Windows scripts

The helper scripts to start and stop the Docker based InfluxDB, Grafana and nginx web-server are provided with _DashT_. The supporting configuration files are installed in `\Users\Public\DashT` folder. Two buttons are installed on the Desktop:

1. Start DB (database _and_ web services)
1. Stop DB

All you need to do to make the buttons work is to **install** yourself [Docker Desktop](https://www.docker.com/products/docker-desktop) and **start it** - following the _DashT_ maxim _"if you do not want it, you will not get it"_, no third party software is installed by _DashT_ installer so that you can keep the control.

Well, basically, that's all! If you are interested in what happens under the hood, you may want to install a Docker management GUI, see [Kitematic](#Kitematic) section.

### Linux scripts

The Linux version of the above is the helper script _**dashtdb**_ - you would use it from the command line:

```
you@yourlinux:~$ dashtdb

dashtdb - Launching Docker based services for DashT: nginx, infludb, grafana
Usage:
      dashtdb [start|up]
      dashtdb [stop|down]
```

>**NOTE**: we let fans of GNOME3, KDE, LXQT, xfce4 and alike to create their own desktop buttons!

The same way, like on the Windows counterpart the principle _"only if you want it you will get it"_ is respected; no third party software is declared as automatic dependency in the _DashT_ package installation. Only if you launch the `dashtdb` script and if the supporting software is not there, you will be asked do you want to get it installed. This way, a person who is not interested in database functions will not get database software installed for nothing.

Third party programs installed by the script - with your permission:

- dashtdb script needs:
  - docker, docker-compose
    - InfluxDB v2.0, Grafana, nginx web server

## Set up InfluxDB

There is no reason to repeat [InfluxDB v2.0 documentation](https://v2.docs.influxdata.com/v2.0/) here. But setting up the database for your boat is dead easy already from the welcome screen of InfluxDB 2.0, which is now at `localhost:9999': the guidance is excellent! You will need to give:

- User name and password
- Organization name - like the name of your boat
- "Bucket" name - this is where the data is going to be dropped into, give it a name like "nmea", why not

If you plan not to use streaming or reading back of data, just file based data storage when underway and its feeding into the database when back in the safe harbor, you are done. You would use, later on this very same interface to upload the data file into the database, into the bucket you have created. Database service itself does not need to run, in this case when you are underway, _DashT_ [InfluxDB Out](../idbout/idbout.ipynb#InfluxDB-Out) is enough.

However, streaming data in live into the database and reading it back live requires a token (your username and password are never asked in data communication):

<img src="img/s_010_DashT_InfluxDB_Tokens.png"
alt="InfluxDB setting a token" width="400">[(zoom)](img/010_DashT_InfluxDB_Tokens.png)

Give the token in [InfluxDB Out](../idbout/idbout.ipynb#HTTP-Streamout) instrument's configuration to enable HTTP-based real-time write/read operations between _DashT_ and InfluxDB v2. You would need to keep the database container running, in this case when underway.

## InfluxDB storage

In this solution with Docker, with _DashT_ configuration scripts data is stored on your computer, the same one which is hosting Docker.

**Windows**: `C:\Users\Public\DashT\influxdb2`

**Linux**: `~/.opencpnplugins/dashboard_tactics_pi/instrujs/influxdb2`

You may want to backup or otherwise keep these directories safe if you want to keep long term archive of the data. _DashT_ cannot guarantee the integrity of the data since this directory is totally under control of Docker/InfluxDB.

>**NOTE**: On Linux you can relink this directory to an external device, such as USB3-connected disk to reduce the load to you system disk against repetitive small chunks of writes (if your system disk a semiconductor storage device) and for backup purposes. (On Windows, Docker will most likely have no permission to write to the device which is out of Public or user's folders.)

## Under the hood

This section you may want to read only when things do not work for you, for simple curiosity or if you want to to the things yourself - maybe you have a Mac and the above scripts are not available for you (but you can maybe contribute by modifying the Linux scripts as a starting point).

### Kitematic

Visual Docker Container Management on Mac & Windows https://kitematic.com (on GitHub https://github.com/docker/kitematic).

<img src="img/s_015_DashT_Kitematic_3_services.png"
alt="Docker - Kitematic - 3 services running" width="200">[(zoom)](img/015_DashT_Kitematic_3_services.png)

View with Kitematic of the three containers created by the above helper scripts, providing three useful services:

* **nginx** - this is the HTTP / proxy server, a swiss knife providing both files to OpenCPN's _DashT_ but also connecting it to other network based services.
* **InfluxDB v2** - time series database which both collects data from DashT but which is also available to feed it back to _DashT_ but also to other useful services:
* **Grafana** - A monitoring solution which allows you to create more complex dashboard's that would be possible with _DashT_ or with _InfluxDB v2_.

>**NOTE**: We do not run a _Signal K server node_ in Docker - it may require some physical connection like USB and it is better done with _Node.js_ which is also network performance wise a better solution than running it in a Docker instance. For testing and learning purposes it is, of course possible to run Signal K as a Docker instance as well.

Command line is the same both for Linux or Windows (and probably for Mac):

`you@yourlinux:~$ docker container ls`

<img src="img/s_020_DashT_Docker_ls_output.png"
alt="Docker - ls-command" width="700">[(zoom)](img/020_DashT_Docker_ls_output.png)

### Container creation

Containers are created using the following type of container definition (example from Windows, in Linux version only the file system mount points do change), the file is named in both `docker-compose.yml`:

```
version: '3'
services:
  web: 
    image: nginx:latest
    container_name: dasht_nginx
    depends_on:
      - db
    volumes:
      - /c/Users/Public/DashT/nginx/nginx.conf:/etc/nginx/nginx.conf
      - /c/Users/Public/DashT/www:/data/www
    ports:
      - 8088:80
      - 8089:8089
  graphs:
    image: grafana/grafana:latest
    container_name: dasht_grafana
    depends_on:
      - db
    links:
      - db
    volumes:
      - /c/Users/Public/DashT/grafana:/var/lib/grafana
    ports:
      - 30000:3000
  db: 
    image: quay.io/influxdb/influxdb:2.0.0-beta
    container_name: dasht_influxdb
    volumes:
      - /c/Users/Public/DashT/influxdb2:/var/lib/influxdb2
    command: influxd run --bolt-path /var/lib/influxdb2/influxd.bolt
    --engine-path /var/lib/influxdb2/engine --store bolt --reporting-disabled
    ports:
      - 9999:9999
```

The description files defines all the steps that one would need to either to type manually or set up using Kitematic. Now it is automatic every time you use the _DashT_ stop/start scripts!

>**NOTE**: On Windows systems, the start-up scripts of _DashT_ also synchronize the InfluxDB container's clock with the local CPU clock. This is needed because on Windows, the clock is provided by Hyper-V virtualization. And when you are not running any containers, that clock is not running... For this reason, it is extremely important to synchronize your CPU time to a reliable data source (like GPS, when underway).

You can observe that both network ports, dependencies and mount points in local file system have been defined. If you need or want to do the work manually, the above description can be used as a starting point for what is needed.

### nginx

[nginx](http://nginx.org/en/) _engine x_ is an HTTP and reverse proxy server. It is a true Swiss knife, allowing us to provide and share all services local, like http://localhost:8088 - we need more services, we just add port numbers. With _nginx_'s rirch features and high its reverse proxy we can

* Make _DashT_ network based files available as network service from the local file system, the same in which OpenCPN is installed - no copying is needed
* We can make the code in those files to access _InfluxDB v2_ **in parallel** with your browser and with _Grafana_ by enabling [CORS](https://en.wikipedia.org/wiki/Cross-origin_resource_sharing) (_Cross-Origin-Resource-Sharing_) so that browser security features gets satisfied

>**NOTE**: without a CORS-enabling proxy it is impossible for a web-based application like JavaScript instruments of _DashT_ to read/write into InfluxDB or other web service since they are not originating from that very same web service. However, [InfluxDB Out](../idbout/idbout.ipynb#InfluxDB-Out) is not affected by this so if you do not use JavaScript instruments, you can omit the CORS-enabling proxy.

We want _nginx_ to configure two local port, to act as proxy server and to deal with CORS so that the localhost served JavaScript files can access services such as the _InfluxDB v2_ - same thing for Grafana, let it access _InfluxDB v2_ :

* port 8088 - is mapped in Docker container as port 80, bind back to the host's local file system so that _DashT_ provided `www` directory (instrument HTML5/JavaScript) can be served
* port 8089 - is a proxy for files served from 8088 wanting to access _influxdb_ server - [CORS Access Control headers](https://en.wikipedia.org/wiki/Cross-origin_resource_sharing#How_CORS_works) are replied to browsers confirming them that this is OK - tested Chrome 80, Firefox 74 and even IE11 (because WebView of wxWidgets is using it).

```
user  nginx;
worker_processes  1;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;


events {
    worker_connections  1024;
}


http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;

    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile        on;
    #tcp_nopush     on;

    keepalive_timeout  65;

    #gzip  on;
    
    server {
      listen 80;
      location / {
        root /data/www;
        autoindex on;
      }
    }
    server {
      listen 8089;
      # https://enable-cors.org/server_nginx.html
      location / {
        # InfluxDB query, read
        if ($request_method = 'OPTIONS') {
            add_header 'Access-Control-Allow-Credentials' 'true' always;
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS';
            add_header Access-Control-Allow-Origin $http_origin;
            add_header 'Access-Control-Allow-Headers'
            'Authorization,Accept,Origin,DNT,User-Agent,X-Requested-With,
            If-Modified-Since,Cache-Control,Content-Type,Range';
            add_header 'Access-Control-Expose-Headers' 'Content-Length,Content-Range';
            #
            # Tell client that this pre-flight info is valid for 20 days
            #
            add_header 'Access-Control-Max-Age' 1728000;
            add_header 'Content-Type' 'text/plain; charset=utf-8';
            add_header 'Content-Length' 0;
            return 204;
        }
        # InfluxDB query, write
        if ($request_method = 'POST') {
            add_header 'Access-Control-Allow-Credentials' 'true' always;
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS';
            add_header 'Access-Control-Allow-Headers' 
            'Authorization,Accept,Origin,DNT,User-Agent,X-Requested-With,
            If-Modified-Since,Cache-Control,Content-Type,Range';
            add_header 'Access-Control-Expose-Headers' 'Content-Length,Content-Range';
        }
        if ($request_method = 'GET') {
            add_header 'Access-Control-Allow-Credentials' 'true' always;
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS';
            add_header 'Access-Control-Allow-Headers'
            'Authorization,Accept,Origin,DNT,User-Agent,X-Requested-With,
            If-Modified-Since,Cache-Control,Content-Type,Range';
            add_header 'Access-Control-Expose-Headers' 'Content-Length,Content-Range';
        }
        proxy_redirect off;
        proxy_set_header host $host;
        proxy_set_header X-real-ip $remote_addr;
        proxy_set_header X-forward-for $proxy_add_x_forwarded_for;
        proxy_pass http://dasht_influxdb:9999;
      }
    }
}
```

>**NOTE**: To read InfluxDB v2.0 database, the client shall use OPTIONS-method, not GET-method which will be rejected. Therefore the CORS-proxy function is implemented only for OPTIONS-method. It is noteworthy also that [InfluxDB Out](../idbout/idbout.ipynb#HTTP-Streamout) will not need to go through the proxy, it can POST directly the data into the TCP/IP port 9999 of the server. But if a JavaScript client needs to do the same, it needs to go through this proxy and the POST-options needs to be implemented with the CORS-proxy functions.

It is nice to be able edit directly the _nginx_'s configuration file without sometimes complicated tricks. That's why we bind it to a local file system file as well.

### Troubleshooting

Troubleshooting of _nginx_ server is best done with an ordinary browser, by attempting to open the aforementioned ports 8088 (you would expect to find _DashT_ JavaScript instrument's home directory to load the HTML5/JavaScript code) and port 8089 (with InfluxDB v2 running on port 9999 you would expect to drop on it's login page).

ALl browser's contain debug tools, for example if you want to open any _DashT_ JavaScript instrument's `index.html` page you can get good information in case of eventual issues with the page loading like that. Please see the [troubleshoot section of EngineDJG](../enginedjg/enginedjg.ipynb#After-config,-all-dead).

In case you do not get connected anywhere, it would be worthwhile to open a console on the Docker container which is providing the _nginx_ service.

>**NOTE**: On **Windows**, you may have left the services running when shutting down, in which case the containers are started automatically. However, one cannot connect to them with Docker tools unless you have also _Docker Desktop_ running. This, you may have selected not to start automatically. If this is the case you need to start it now. If you are using, as suggested [Kitematic](#Kitematic) you have the console available on it and you do not need necessarily the below commands.

Check first that you have, indeed the containers still running:
```
docker container ls
```

From command line, attach the console to the _dasht_nginx_ container:
```
docker container attach dasht_nginx
```

Try for the above connections again and observe the console. It should say, if working correctly access to all ports (or error messages if an issue):
```
172.20.0.1 - - [07/Sep/2020:09:54:59 +0000] "GET / HTTP/1.1" 200
1301 "-" "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36
(KHTML, like Gecko) Chrome/85.0.4183.106 Safari/537.36" "-"
172.20.0.1 - - [07/Sep/2020:09:55:00 +0000] "GET /favicon.ico
HTTP/1.1" 200 1150 "http://localhost:8088/" "Mozilla/5.0 (Windows
NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko)
Chrome/85.0.4183.106 Safari/537.36" "-"
172.20.0.1 - - [07/Sep/2020:09:55:04 +0000] "GET
/orgs/59f148cfc72908cc HTTP/1.1" 200 313 "-" "Mozilla/5.0 (Windows
NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko)
Chrome/85.0.4183.106 Safari/537.36" "-"
172.20.0.1 - - [07/Sep/2020:09:55:04 +0000] "GET /5e93c5f5aa.js
HTTP/1.1" 304 0 "http://localhost:8089/orgs/59f148cfc72908cc"
"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML,
like Gecko) Chrome/85.0.4183.106 Safari/537.36" "-"
...
```

Stop the console attachment with `Ctrl-P` `Ctrl-Q` key-combination - this way the container will continue running. To get back to command prompt after this, use `Ctrl-C` as usual.

### Grafana

[Grafana](https://grafana.com) is an open source analytics and monitoring solution for almost every database based data, including _InfluxDB v2.0_. _DashT_ bundles this popular and easy to use visualization solution it in its Docker ready-to-launch configuration albeit it does not use it, in any way in _OpenCPN_.

With _Grafana_ we want to use a separated volume to share it settings and other parameters in persistent manner, provided that we need to delete and restart the container. The volume is located in local file system as defined in `docker-compose.yml` (see above).